# Non-Linear Geostatistics

This case study is meant to demonstrate how to use *gstlearn* for non-linear geostatistics.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
return false;
}

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np

Global parameters

In [ ]:
# Set the Seed for the Random Number generator
gl.law_set_random_seed(32131)

# Define the Space Dimension
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim)

# Defining the format for dumping statistics of variables in a Db
dbfmt = gl.DbStringFormat()
dbfmt.setFlags(flag_resume=True,flag_vars=False,flag_locator=True)

# Defining global options
flagDebugCalculations = True
flagPerformGraphics = True

Setting the trace: this option alloes dumping all the elements to check calculations when the rank of the target is equal to 1 (first block, first panel, ...). This option is important for debugging but it creates lots of printout.

In [ ]:
if flagDebugCalculations:
    gl.OptDbg.setReference(1)

Defining color scales common to all future representations of estimated quantities:

In [ ]:
ZEstMin = -3.
ZEstMax = +3.
TEstMin = 0.
TEstMax = 1.
TStdMin = 0.
TStdMax = 1.
QEstMin = 0.
QEstMax = 3.
QStdMin = 0.
QStdMax = 2.

Generate initial grid

In [ ]:
grid = gl.DbGrid.create([100,100], [0.01,0.01])
grid.display()

Create grid of Panels covering the entiere simulated area

In [ ]:
dx_P = 0.25
panel = gl.DbGrid.createCoveringDb(grid,dcell=[dx_P,dx_P])

Discretization with a grid of blocks which cover the simulated area

In [ ]:
nx_B = 5
dx_B = dx_P / nx_B
blocs = gl.DbGrid.createCoveringDb(grid,dcell=[dx_B,dx_B])
blocs.display()

Simulation of the Data

In [ ]:
model_init = gl.Model.createFromParam(gl.ECov.EXPONENTIAL, range=0.1, sill=1.)
gl.simtub(None, grid, model_init)
grid.setName("Simu", "Y")
grid.display()

Non-linear transform

In [ ]:
m_Z = 1.5
s_Z = 0.5
Zval = np.array(grid.getColumn("Y"))
for i in range(grid.getSampleNumber()):
    Zval[i] = m_Z * np.exp(s_Z * Zval[i] - 0.5 * s_Z * s_Z)
iuid = grid.addColumns(Zval, "Z")

In [ ]:
if flagPerformGraphics:
    ax = gp.grid(grid, name="Z", aspect=1)

Data extraction

In [ ]:
np = 500
data = gl.Db.createSamplingDb(grid, number=np, names=["x1","x2","Y","Z"])
data.setLocator("Z", gl.ELoc.Z)
data.display(dbfmt)

In [ ]:
if flagPerformGraphics:
    ax = gp.point(data, color_name="Z", size_name="Z", 
                  flagColorBar=False, flagSizeLegend=False, aspect=1)

Gaussian Anamorphosis with 10 coefficients

In [ ]:
anam = gl.AnamHermite.create(nbpoly=20)
anam.fitFromLocator(data)
anam.display()

In [ ]:
if flagPerformGraphics:
    ax = gp.anam(anam)

Selectivity: We focus on Tonnage (T) and Metal Quantity (Q) for 2 cutoffs (0 and 0.5)

In [ ]:
selectivity = gl.Selectivity_createByCodes([gl.ESelectivity.Q, gl.ESelectivity.T], zcuts=[0., 0.5], 
                                           flag_est=True, flag_std=True)

Global experimental selectivity, calculated form the experimental Data Set

In [ ]:
selectivity.eval(data)

Selectivity in the model, derived from the parameters contained in the Anamorphosis

In [ ]:
selectivity.eval(anam)

Define the variogram calculation parameters: omni-directional variogram calculated in 2-D pace, with 10 lags of 0.025.

In [ ]:
varioparam = gl.VarioParam_createOmniDirection(npas=10, dpas=0.025)

Calculate the variogram of the raw variable

In [ ]:
vario_raw = gl.Vario_computeFromDb(varioparam, db=data)

Fitting the Model on the Raw variable

In [ ]:
model_raw = gl.Model_createFromEnvironment(nvar=1, ndim=ndim)
model_raw.fit(vario_raw)
model_raw.display()

In [ ]:
if flagPerformGraphics:
    ax = gp.varmod(vario_raw, model_raw)

Transform Data into Gaussian variable

In [ ]:
gl.RawToGaussianByLocator(data, anam=anam)
data.setName("Y.Z","Gauss.Z")
data.display()

Calculate the variogram of the Gaussian variable

In [ ]:
vario = gl.Vario_computeFromDb(varioparam, data)

Fitting the Model on the Gaussian transformed variable (under the constraint that the sill must be equal to 1)

In [ ]:
model = gl.Model(1, ndim)
constraints = gl.Constraints(1.)
model.fit(vario, [gl.ECov.EXPONENTIAL, gl.ECov.EXPONENTIAL], 
          mauto=gl.Option_AutoFit(), constraints=constraints)
model

In [ ]:
if flagPerformGraphics:
    ax = gp.varmod(vario, model)

Creating a Moving Neighborhood

In [ ]:
nmini = 5
nmaxi = 5
radius = 1.
neigh = gl.NeighMoving.create(nmaxi=nmaxi, radius=radius, nmini=nmini)
neigh

## ====== Conditional Expectation ======

Estimating the Gaussian Variable on the nodes of the Blocks

In [ ]:
data.setLocator("Gauss.Z",gl.ELoc.Z)
dummy = gl.kriging(data, blocs, model, neigh, calcul=gl.EKrigOpt.PONCTUAL,
        namconv=gl.NamingConvention("G_PTS"))

In [ ]:
if flagPerformGraphics:
    ax = gp.grid(blocs, vmin=ZEstMin, vmax=ZEstMax, aspect=1)

Calculating the Conditional Expectation on blocks

In [ ]:
dummy = gl.ConditionalExpectation(blocs, anam, selectivity, 
                                  name_est="G_PTS*estim", name_std="G_PTS*stdev", 
                                  namconv = gl.NamingConvention("PTS_Recovery", False))

In [ ]:
blocs

In [ ]:
axs = gp.grids(blocs, "PTS_Recovery.Q-estim*",figsize=[9,9], aspect=1,
               vmin=QEstMin, vmax=QEstMax)

In [ ]:
axs = gp.grids(blocs, "PTS_Recovery.Q-stdev*",figsize=[9,9], aspect=1,
               vmin=QStdMin, vmax=QStdMax)

In [ ]:
axs = gp.grids(blocs, "PTS_Recovery.T-estim*",figsize=[9,9], aspect=1,
               vmin=TEstMin, vmax=TEstMax)

In [ ]:
axs = gp.grids(blocs, "PTS_Recovery.T-stdev*",figsize=[9,9], aspect=1,
               vmin=TStdMin, vmax=TStdMax)

## ====== Point Disjunctive Kriging ======

Attach the Anamorphosis to the Model

In [ ]:
dummy = model.setAnam(anam)

Computing the Point factors

In [ ]:
nfactor = 3
dummy = gl.RawToFactor(data, anam, nfactor)

Simple Point Kriging over the blocks

In [ ]:
dummy = gl.DisjunctiveKriging(data, blocs, model, neigh, calcul=gl.EKrigOpt.PONCTUAL,
                              namconv = gl.NamingConvention("DK_Pts"))
blocs.display()

In [ ]:
axs = gp.grids(blocs,names="DK_Pts*estim",figsize=[15,15], aspect=1)

In [ ]:
axs = gp.grids(blocs,names="DK_Pts*stdev",figsize=[15,15], aspect=1)

Simple Block Kriging over the blocks

In [ ]:
ndisc_B = [5,5]
dummy = gl.DisjunctiveKriging(data, blocs, model, neigh, 
                              calcul=gl.EKrigOpt.BLOCK, ndisc=ndisc_B, 
                              namconv = gl.NamingConvention("DK_Blk"))
blocs.display()

In [ ]:
axs = gp.grids(blocs,names="DK_Blk*estim",figsize=[15,15], aspect=1)

In [ ]:
axs = gp.grids(blocs,names="DK_Blk*stdev",figsize=[15,15], aspect=1)

Comparing Point and Block estimations and standard deviation of Estimation errors (this comparison is performed on the results of the first factor only).

In [ ]:
if flagPerformGraphics:
    ax = gp.correlation(blocs, "DK_Pts*1.estim", "DK_Blk*1.estim")

In [ ]:
if flagPerformGraphics:
    ax = gp.correlation(blocs, "DK_Pts*1.stdev", "DK_Blk*1.stdev")

Simple Block Kriging over the panel(s)

In [ ]:
ndisc_P = [10,10]
dummy = gl.DisjunctiveKriging(data, panel, model, neigh, 
                              calcul=gl.EKrigOpt.BLOCK, ndisc=ndisc_P,
                              namconv=gl.NamingConvention("DK_Blk"))
panel.display()

In [ ]:
axs = gp.grids(panel,names="DK_Blk*estim",figsize=[15,15], aspect=1)

In [ ]:
axs = gp.grids(panel,names="DK_Blk*stdev",figsize=[15,15], aspect=1)

## ===== Uniform Conditioning =====

Calculate the Mean covariance over block

In [ ]:
cvv_Z = model_raw.evalCvv(blocs.getDXs(), ndisc=ndisc_B)

Perform the Point Kriging of the Raw Variable

In [ ]:
data.clearLocators(gl.ELoc.Z)
data.setLocator("Z",gl.ELoc.Z)
dummy = gl.kriging(data, blocs, model, neigh, calcul=gl.EKrigOpt.PONCTUAL,
                   namconv=gl.NamingConvention("Z_PTS"))
blocs.display()

In [ ]:
axs = gp.grids(blocs,names="Z_PTS*",figsize=[9,9], aspect=1)

Perform the Uniform Conditioning over Blocks

In [ ]:
dummy = gl.UniformConditioning(blocs, anam, selectivity,
                               names_est="Z_PTS*estim", names_std="Z_PTS*stdev", cvv=cvv_Z, 
                               namconv = gl.NamingConvention("UC",False))
blocs.display()
data.setLocator("Gauss.Z",gl.ELoc.Z)

In [ ]:
axs = gp.grids(blocs, "UC.Q*",figsize=[9,9], aspect=1)

In [ ]:
axs = gp.grids(blocs, "UC.T*",figsize=[6,6], aspect=1)

## ===== Block Disjunctive Kriging (DGM-1) =====

Calculate the change of support coefficient

In [ ]:
r1 = anam.evalSupportCoefficient(1, model, dxs=blocs.getDXs(), ndisc=ndisc_B)

Update the Model with Block anamorphosis

In [ ]:
anam_b1 = anam.clone()
anam_b1.setRCoef(r1)

Regularization of the point model by the block support

In [ ]:
vario_b1_Z = gl.Vario_createRegularizeFromModel(model, varioparam, 
                                                ext=blocs.getDXs(),
                                                ndisc=ndisc_B, 
                                                angles=blocs.getAngles())
cvv = model.evalCvv(blocs.getDXs(), ndisc_B)
vario_b1_Y = gl.Vario_createTransformZToY(vario_b1_Z, anam, cvv)

In [ ]:
if flagPerformGraphics:
    ax = gp.vario(vario_b1_Z)

Fitting the regularized model on the point Gaussian variable

In [ ]:
model_b1_Y = gl.Model(1, ndim)
constraints.setConstantSillValue(1)
dummy = model_b1_Y.fit(vario_b1_Y, types=[gl.ECov.CUBIC, gl.ECov.EXPONENTIAL],
                       mauto=gl.Option_AutoFit(), constraints=constraints)

In [ ]:
ax = gp.varmod(vario_b1_Y, model_b1_Y)

Update the Model with Block Anamorphosis

In [ ]:
model_b1_Y.setAnam(anam_b1)
model_b1_Y.display()

Simple Point Kriging over the blocs(s) with Model with Change of Support

In [ ]:
dummy = gl.DisjunctiveKriging(data, blocs, model_b1_Y, neigh, gl.EKrigOpt.PONCTUAL,
                              [], True, True,
                              gl.NamingConvention("DK_DGM1"))
blocs.display()

In [ ]:
axs = gp.grids(blocs, names="DK_DGM1*", figsize=[9,9], aspect=1)

Simple Point Kriging over the panel(s) with Model with Change of Support

In [ ]:
dummy = gl.DisjunctiveKriging(data, panel, model_b1_Y, neigh, 
                              calcul=gl.EKrigOpt.BLOCK, ndisc=[nx_B, nx_B], 
                              namconv=gl.NamingConvention("DK_DGM1"))
panel.display()

In [ ]:
axs = gp.grids(panel, names="DK_DGM1*", figsize=[9,9], aspect=1)

## ===== Block Disjunctive Kriging (DGM-2) =====

Calculate the change of support coefficient

In [ ]:
r2 = anam.evalSupportCoefficient(2, model, dxs=blocs.getDXs(), ndisc=ndisc_B)

Update the Model with Block anamorphosis

In [ ]:
anam_b2 = anam.clone()
anam_b2.setRCoef(r2)

Regularization of the point model by the block support

In [ ]:
vario_b2_Y = gl.Vario_createRegularizeFromModel(model, varioparam, 
                                                ext = blocs.getDXs(),
                                                ndisc = ndisc_B, 
                                                angles = blocs.getAngles())

In [ ]:
ax = gp.vario(vario_b2_Y)

Fitting the regularized model on the point Gaussian variable

In [ ]:
model_b2_Y = gl.Model(1, ndim)
constraints.setConstantSillValue(r2 * r2)
dummy = model_b2_Y.fit(vario_b2_Y, types=[gl.ECov.CUBIC, gl.ECov.EXPONENTIAL], 
                       mauto = gl.Option_AutoFit(), constraints = constraints)

In [ ]:
ax = gp.varmod(vario_b2_Y, model_b2_Y)

Normalization of the block model to a total sill equal to 1.0

In [ ]:
model_b2_Y.normalize(1.0)
model_b2_Y.display()

In [ ]:
ax = gp.model(model_b2_Y, hmax=vario_b2_Y.getHmax())

Update the Model with Block anamorphosis

In [ ]:
model_b2_Y.setAnam(anam_b2)
model_b2_Y.display()

Simple Point Kriging over the blocs(s) with Model with Change of Support

In [ ]:
dummy = gl.DisjunctiveKriging(data, blocs, model_b2_Y, neigh, calcul=gl.EKrigOpt.PONCTUAL,
                              namconv=gl.NamingConvention("DK_DGM2"))
blocs.display()

In [ ]:
axs = gp.grids(blocs, names="DK_DGM2*", figsize=[9,9], aspect=1)

Simple Point Kriging over the panel(s) with Model with Change of Support

In [ ]:
dummy = gl.DisjunctiveKriging(data, panel, model_b2_Y, neigh, 
                              calcul=gl.EKrigOpt.BLOCK, ndisc=[nx_B, nx_B],
                              namconv=gl.NamingConvention("DK_DGM2"))
panel.display()

In [ ]:
axs = gp.grids(panel, names="DK_DGM2*", figsize=[9,9], aspect=1)

## ===== Selectivity Function =====

In [ ]:
gl.FactorToSelectivity(blocs, anam, selectivity, 
                       names_est = blocs.getNames("DK_Pts*estim"),
                       names_std = blocs.getNames("DK_Pts*stdev"), 
                       namconv=gl.NamingConvention("DK_Pts_Sel", False))
blocs.display()

In [ ]:
axs = gp.grids(blocs, "DK_Pts_Sel.T*",figsize=[6,6], aspect=1)

In [ ]:
axs = gp.grids(blocs, "DK_Pts_Sel.Q*",figsize=[6,6], aspect=1)